In [1]:
import json
import requests
import csv
import math
import time
from bs4 import BeautifulSoup
import pandas as pd
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
import nltk
import re
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to C:\Users\Navaneeth
[nltk_data]     kishore\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Navaneeth
[nltk_data]     kishore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Navaneeth
[nltk_data]     kishore\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
access_pt = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
api_key = "zYJsAQ968WPrEC01QcoXCUUkn71n9XKI"

In [3]:
topic = "music"
subtopics = ["pop","jazz","classical music","rock","rap","country"]

In [4]:
stop_list = set(stopwords.words("english"))
article_dict = {}

In [5]:
begin_date = "20190101"
end_date = "20190415"
fl = "web_url, _id"
fq = "subsection_name:\"" + topic + "\" OR subject: \"" + topic + "\""
q = topic
q_params = {'q':q, 'api-key':api_key, 'fl':fl, 'fq':fq, 'begin_date':begin_date, 'end_date':end_date}
res = requests.get(url = access_pt, params = q_params)
json_res = res.json()
num_hits = json_res['response']['meta']['hits']
response_docs = json_res['response']['docs']

with open(topic + ".txt", "a+", encoding="utf-8") as text_file:
    for a in response_docs:
        _id = a['_id']
        url = a['web_url']
        if _id not in article_dict and url != "":
            url_res = requests.get(url)
            doc = BeautifulSoup(url_res.text, "html.parser")
            article = ""
            act_article = ""
            for div in doc.find_all("div", class_="StoryBodyCompanionColumn"):
                for paragraph in div.find_all('p'):
                    act_article = act_article + paragraph.get_text() + " "
                    paragraph = " ".join(re.findall("[a-zA-Z]+",paragraph.get_text().lower()))
                    paragraph_text = word_tokenize(paragraph)
                    final_text = ''.join([WordNetLemmatizer().lemmatize(w) + " " for w in paragraph_text if not w in stop_list])
                    article = article + final_text + "\n"
            article_dict[_id] = [url, act_article]
            text_file.write(article)

num_pages = math.ceil(num_hits/10)
page = 0

with open(topic + ".txt", "a+", encoding="utf-8") as text_file:
    while(num_pages > page):
        q_params = {'q':q, 'api-key':api_key, 'fl':fl, 'fq':fq, 'page':page, 'begin_date':begin_date, 'end_date':end_date}
        local_res = requests.get(url = access_pt, params = q_params)
        local_json_res = local_res.json()
        local_response_docs = local_json_res['response']['docs']
        for a in local_response_docs:
            _id = a['_id']
            url = a['web_url']
            if _id not in article_dict and url != "":
                url_res = requests.get(url)
                doc = BeautifulSoup(url_res.text, "html.parser")
                article = ""
                act_article = ""
                for div in doc.find_all("div", class_="StoryBodyCompanionColumn"):
                    for paragraph in div.find_all('p'):
                        act_article = act_article + paragraph.get_text() + " "
                        paragraph = " ".join(re.findall("[a-zA-Z]+", paragraph.get_text().lower()))
                        paragraph_text = word_tokenize(paragraph)
                        final_text = ''.join([WordNetLemmatizer().lemmatize(w) + " " for w in paragraph_text if not w in stop_list])
                        article = article + final_text + "\n"
                article_dict[_id] = [url, act_article]
                text_file.write(article)
        time.sleep(6)
        page += 1

print("DONE WITH " + topic)

DONE WITH music


In [6]:
for q in subtopics:
    begin_date = "20190101"
    end_date = "20190415"
    fl = "web_url, _id"
    fq = "subsection_name :\"" + topic + "\" OR subject : \"" + topic + "\" OR subject : \"" + q + "\""
    q_params = {'q':q, 'api-key':api_key, 'fl':fl, 'fq':fq, 'begin_date':begin_date, 'end_date':end_date}
    res = requests.get(url = access_pt, params = q_params)
    json_res = res.json()
    num_hits = json_res['response']['meta']['hits']
    response_docs = json_res['response']['docs']
    q_dict = {}

    with open(q + ".txt", "a+", encoding="utf-8") as text_file:
        for a in response_docs:
            _id = a['_id']
            url = a['web_url']
            if _id not in q_dict and url != "":
                url_res = requests.get(url)
                doc = BeautifulSoup(url_res.text, "html.parser")
                article = ""
                act_article = ""
                for div in doc.find_all("div", class_="StoryBodyCompanionColumn"):
                    for paragraph in div.find_all('p'):
                        act_article = act_article + paragraph.get_text() + " "
                        paragraph = " ".join(re.findall("[a-zA-Z]+",paragraph.get_text().lower()))
                        paragraph_text = word_tokenize(paragraph)
                        final_text = ''.join([WordNetLemmatizer().lemmatize(w) + " " for w in paragraph_text if not w in stop_list])
                        article = article + final_text + "\n"
                q_dict[_id] = [url, act_article]
                text_file.write(article)
                if _id not in article_dict:
                    article_dict[_id] = [url, act_article]
                

    num_pages = math.ceil(num_hits/10)
    page = 0
    
    with open(q + ".txt", "a+", encoding="utf-8") as text_file:
        while(num_pages > page):
            q_params = {'q':q, 'api-key':api_key, 'fl':fl, 'fq':fq, 'page':page, 'begin_date':begin_date, 'end_date':end_date}
            local_res = requests.get(url = access_pt, params = q_params)
            local_json_res = local_res.json()
            local_response_docs = local_json_res['response']['docs']
            for a in local_response_docs:
                _id = a['_id']
                url = a['web_url']
                if _id not in q_dict and url != "":
                    url_res = requests.get(url)
                    doc = BeautifulSoup(url_res.text, "html.parser")                
                    article = ""
                    act_article = ""
                    for div in doc.find_all("div", class_="StoryBodyCompanionColumn"):
                        for paragraph in div.find_all('p'):
                            act_article = act_article + paragraph.get_text() + " "
                            paragraph = " ".join(re.findall("[a-zA-Z]+",paragraph.get_text().lower()))
                            paragraph_text = word_tokenize(paragraph)
                            final_text = ''.join([WordNetLemmatizer().lemmatize(w) + " " for w in paragraph_text if not w in stop_list])
                            article = article + final_text + "\n"
                    q_dict[_id] = [url, act_article]
                    text_file.write(article)
                    if _id not in article_dict:
                        article_dict[_id] = [url, act_article]
            time.sleep(6)
            page += 1
    
    qdf = pd.DataFrame.from_dict(q_dict, orient="index")
    qdf.to_csv(q+".csv")
    print("DONE WITH SUBTOPIC " + q)

DONE WITH SUBTOPIC pop
DONE WITH SUBTOPIC jazz
DONE WITH SUBTOPIC classical music
DONE WITH SUBTOPIC rock
DONE WITH SUBTOPIC rap
DONE WITH SUBTOPIC country


In [7]:
df = pd.DataFrame.from_dict(article_dict, orient="index")
df.to_csv("articles.csv")